### Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines


In [6]:
#!pip install openai==0.28.1
#!pip install openai --upgrade
#!pip install ragas
#!pip install unstructured
#!pip install langchain[all]
#!pip install --upgrade langchain

#!pip install playwright
#!pip install -U selenium unstructured
#!pip install --upgrade langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [1]:
#!pip install pydantic==2.5
#!pip install rapidocr-onnxruntime

In [1]:
#!pip install nltk

In [1]:
import os, json
from langchain.chains import AnalyzeDocumentChain
from langchain_openai import ChatOpenAI

from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 
from llm_utils import load_pdf_documets, RecursiveCharacterTextSplitter
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

#openai.api_key = os.environ['OPENAI_API_KEY']

[nltk_data] Downloading package punkt_tab to /home/oleg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
#from langchain_community.document_loaders import SeleniumURLLoader,  DirectoryLoader, PyPDFLoader
#from langchain.text_splitter import CharacterTextSplitter,  RecursiveCharacterTextSplitter

In [3]:
from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, top_p=0.2)  

/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


In [31]:


# Specify the directory
directory = './data/data_rag/conference/'

# List all files in the directory
pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

# Print PDF file names
#print(pdf_list)
pdf_files = [os.path.join(directory, f) for f in pdf_files]


chunks, documents = load_pdf_documets(pdf_files, tokens=5000)
print(len(chunks), len(documents))

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong poi

163 163


### Topics of documents

In [5]:
from prompt_utills import *
from time import time
from langchain_core.output_parsers import JsonOutputParser

In [11]:
extract_topic = PromptTemplate.from_template(
            """
            You are an expert in analyzing scientific papers.
            You have received the text:
            ###
                {text}
            ------------- \n
            Your task is to carefully analyze this text. 
            And answer the question:
            What are the main subject areas of research and study in the text?
            Compare these subject areas with the names in the list:
            ###
               {topics}
            --------------\n
            Name more than one, but less than {N} names of these areas or topics. 
            If possible, use the names from the list, otherwise, 
            if the text contains a name of the subject of research that does not exist in the list, give a new name of the topic.
            
            The output format is a dictionary in valid JSON format.
            Where the keys are the names of the topics studied in the text, and the values are short descriptions of these topics.
            Example: 
                <Topic_title1 : short_description_1, 
                Topic_title2 : short_description_2, 
                ...
                    >
            """
                )

In [32]:
def merge_dict(dict1, dict2):
    for key in dict2.keys():
        if key in dict1:
            dict1[key]['metadata'] = dict1[key]['metadata'] + dict2[key]['metadata']
        else:
            dict1[key] = dict2[key]
    return dict1


def rename_dict_key(dict_in, dict_rename):
    for k,v in dict_rename.items():
        if k != v['name']:
            if v['name'] not in  dict_in.keys():
                dict_in[v['name']] = dict_in[k]
                del dict_in[k]
            else: 
                dict_in[v['name']]['description'] = v['description']
                dict_in[v['name']]['metadata'] = dict_in[v['name']]['metadata'] + dict_in[k]['metadata']
                del dict_in[k]
    return dict_in


def add_metadata(doc, dict_t, dict_rename):
    
    def _get_v(k):
        o = dict_rename.get(k, k)
        return o['name'] if type(o) == dict else k

    l_topics = list(set([_get_v(k) for k in dict_t.keys()]))
    if len(doc.metadata.get('topic', [])):
        doc.metadata['topic'] = [_get_v(k) for k in doc.metadata['topic']]
    else:
        doc.metadata['topic'] = l_topics
       
    return doc

def clear_str(s, p="```"):
    i = s.find(p)
    return s[i:]

def check_rename(dict_topics, rename):
    
    check = True
    max_it = 4
    while check:
        # Checking one-to-one correspondence
        _topics = llm.invoke(
                    check_topic.format(list2=sorted(dict_topics.keys()), 
                                       list1=sorted(rename.keys()))
                    ).content
        _topics = clear_str(_topics)
        _topics = _topics.strip("```json").strip("```").strip()
        try:
            _rename = json.loads(_topics) # Checking dictionary
            check = False if len(_rename) == len(rename) else True
        except:
            check = True
        max_it -=1
        if max_it < 0:
            return rename
    return {_rename[k]: v for k,v in rename.items()}


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser()

extract_topic.partial_variables={"format_instructions": parser.get_format_instructions()}
check_dublicates_topic.partial_variables={"format_instructions": parser.get_format_instructions()}


# Extracting topics
dict_topics = {} # extracted topics dictionary

for i, doc in enumerate(documents):
    
    t0 = time()
    
    # Extract topics from a document
    chain = extract_topic | llm | parser 
    _dict = chain.invoke({"N": 10, 
                          "text":doc.page_content, 
                          "topics":sorted(dict_topics.keys())})
    
    print(f"DocN {i}, time {time() - t0}, doc.metadata {doc.metadata}")
    
    # The dictionary of topics with metadata
    _dict = {k: {'description': v,
                'metadata': [doc.metadata],
                }
                   for k, v in _dict.items()
            } 
    
    # Merge with existing dictionary of topics.
    dict_topics = merge_dict(dict_topics, _dict)
    old_len = len(dict_topics)
    
    # Checking the topic dictionary on dublicates
    tE = time()
    topic_list = {k: v['description'] for k,v in dict_topics.items()}
    
    # Check topic's names on dublicates
    chain = check_dublicates_topic | llm | parser
    rename = chain.invoke({"topics": dict_topics})
    
    # Add topics as metadata to the document
    doc = add_metadata(doc, _dict, rename)
    
    # Rename topics in previous documents
    for j in range(i): 
        chunks[j] = add_metadata(chunks[j], _dict, rename)
    
    dict_topics = rename_dict_key(dict_topics, rename)
    
    print(f"OLD len: {old_len} NEW len: {len(dict_topics)} \n time Checking {time()-tE} All {time()-t0}")
    print()
    
    

DocN 0, time 2.6024105548858643, doc.metadata {'source': './data/data_rag/conference/45_AI4AM2024_Hakim_AMARA_8.pdf', 'page': 0}
OLD len: 5 NEW len: 5 
 time Checking 4.208849191665649 All 6.8114540576934814

DocN 1, time 3.0650200843811035, doc.metadata {'source': './data/data_rag/conference/35_AI4AM2024_Cheng.pdf', 'page': 0}
OLD len: 10 NEW len: 7 
 time Checking 7.1614274978637695 All 10.22663927078247

DocN 2, time 1.3287053108215332, doc.metadata {'source': './data/data_rag/conference/4_AI4AM2024_Magdau_Ioan_Bogdan_39.pdf', 'page': 0}
OLD len: 7 NEW len: 6 
 time Checking 4.626109838485718 All 5.954965114593506

DocN 3, time 1.9300837516784668, doc.metadata {'source': './data/data_rag/conference/4_AI4AM2024_Magdau_Ioan_Bogdan_39.pdf', 'page': 0}
OLD len: 7 NEW len: 7 
 time Checking 5.649213552474976 All 7.579489707946777

DocN 4, time 1.5112862586975098, doc.metadata {'source': './data/data_rag/conference/4_AI4AM2024_Magdau_Ioan_Bogdan_39.pdf', 'page': 0}
OLD len: 7 NEW len: 7 


OLD len: 11 NEW len: 11 
 time Checking 10.763202905654907 All 12.58113408088684

DocN 39, time 1.7288696765899658, doc.metadata {'source': './data/data_rag/conference/54_AI4AM2024_Ustyuzhanin_Andrey_64.pdf', 'page': 0}
OLD len: 12 NEW len: 12 
 time Checking 11.189679861068726 All 12.918741941452026

DocN 40, time 2.426405191421509, doc.metadata {'source': './data/data_rag/conference/54_AI4AM2024_Ustyuzhanin_Andrey_64.pdf', 'page': 0}
OLD len: 12 NEW len: 12 
 time Checking 8.327766418457031 All 10.754331827163696

DocN 41, time 2.841806173324585, doc.metadata {'source': './data/data_rag/conference/10_AI4AM2024_Khatibi_Zahra_74.pdf', 'page': 0}
OLD len: 12 NEW len: 12 
 time Checking 10.03612232208252 All 12.87804102897644

DocN 42, time 4.612526178359985, doc.metadata {'source': './data/data_rag/conference/60_AI4AM2024_Marco_Moors_50.pdf', 'page': 0}
OLD len: 12 NEW len: 12 
 time Checking 10.03911828994751 All 14.651886224746704

DocN 43, time 2.245237350463867, doc.metadata {'sourc

OLD len: 24 NEW len: 24 
 time Checking 18.563462018966675 All 21.015393495559692

DocN 77, time 1.8029985427856445, doc.metadata {'source': './data/data_rag/conference/91_AI4AM2024_Wenzel.pdf', 'page': 0}
OLD len: 26 NEW len: 26 
 time Checking 23.605823278427124 All 25.408904790878296

DocN 78, time 2.6361801624298096, doc.metadata {'source': './data/data_rag/conference/75_AI4AM2024_Pathak_Swapneel_Amit_51.pdf', 'page': 0}
OLD len: 26 NEW len: 26 
 time Checking 21.853909015655518 All 24.49023747444153

DocN 79, time 2.4906654357910156, doc.metadata {'source': './data/data_rag/conference/83_AI4AM2024_Delgado_Galindo_Pedro_Julian_87.pdf', 'page': 0}
OLD len: 26 NEW len: 26 
 time Checking 23.653380155563354 All 26.144158601760864

DocN 80, time 3.2079715728759766, doc.metadata {'source': './data/data_rag/conference/64_AI4AM2024_Vozza_Mario_59.pdf', 'page': 0}
OLD len: 26 NEW len: 26 
 time Checking 39.145031452178955 All 42.35321307182312

DocN 81, time 1.8061327934265137, doc.metadat

OLD len: 45 NEW len: 45 
 time Checking 44.40564465522766 All 47.45997977256775

DocN 115, time 2.907871961593628, doc.metadata {'source': './data/data_rag/conference/33_AI4AM2024_Alducin.pdf', 'page': 0}
OLD len: 46 NEW len: 46 
 time Checking 47.453471660614014 All 50.36157512664795

DocN 116, time 2.3198792934417725, doc.metadata {'source': './data/data_rag/conference/87_AI4AM2024_Abio_Albert_88.pdf', 'page': 0}
OLD len: 47 NEW len: 47 
 time Checking 47.691256284713745 All 50.01131463050842

DocN 117, time 2.0776357650756836, doc.metadata {'source': './data/data_rag/conference/87_AI4AM2024_Abio_Albert_88.pdf', 'page': 0}
OLD len: 47 NEW len: 47 
 time Checking 72.36203241348267 All 74.43985724449158

DocN 118, time 2.5774848461151123, doc.metadata {'source': './data/data_rag/conference/87_AI4AM2024_Abio_Albert_88.pdf', 'page': 1}
OLD len: 47 NEW len: 47 
 time Checking 48.99385190010071 All 51.57139849662781

DocN 119, time 2.0121469497680664, doc.metadata {'source': './data/data_r

OLD len: 61 NEW len: 61 
 time Checking 53.18191599845886 All 55.99414873123169

DocN 153, time 2.04950213432312, doc.metadata {'source': './data/data_rag/conference/90_AI4AM2024_Schleder_Gabriel_42.pdf', 'page': 1}
OLD len: 61 NEW len: 61 
 time Checking 48.71904993057251 All 50.768832206726074

DocN 154, time 2.554095983505249, doc.metadata {'source': './data/data_rag/conference/3_AI4AM2024_Shibaev_Egor_66.pdf', 'page': 0}
OLD len: 61 NEW len: 61 
 time Checking 51.26036286354065 All 53.81462478637695

DocN 155, time 2.0767099857330322, doc.metadata {'source': './data/data_rag/conference/3_AI4AM2024_Shibaev_Egor_66.pdf', 'page': 0}
OLD len: 61 NEW len: 61 
 time Checking 53.022496700286865 All 55.099448680877686

DocN 156, time 2.1859376430511475, doc.metadata {'source': './data/data_rag/conference/66_AI4AM2024_Benaissa_Mohammed_62.pdf', 'page': 0}
OLD len: 63 NEW len: 63 
 time Checking 49.230072021484375 All 51.41621112823486

DocN 157, time 2.0713882446289062, doc.metadata {'sourc

In [33]:
len(dict_topics), sorted(dict_topics.keys())
    

(68,
 ['3D Printing',
  'Active Learning',
  'Additive Manufacturing',
  'Anomaly Detection',
  'Band Gap Engineering',
  'Bayesian Optimization',
  'Biosensors',
  'Catalysis',
  'Charge Mixing Optimization',
  'Computational Chemistry and Materials Science',
  'Computational Materials Science',
  'Crystal Structure Determination',
  'Curriculum Learning',
  'Data Acquisition and Processing',
  'Data Analysis',
  'Deep Learning',
  'Density Functional Approximations (DFAs)',
  'Density Functional Theory (DFT)',
  'Dimensional Analysis',
  'Elastic Energy Calculation',
  'Electrochemical Devices',
  'Electroencephalography (EEG)',
  'Exchange-Correlation Functionals',
  'Explainable Artificial Intelligence (XAI)',
  'Ferroic Materials',
  'Finite Element Method (FEM)',
  'Gaussian Process Algorithms',
  'Graph Neural Networks',
  'High-Entropy Alloys (HEAs)',
  'Interatomic Potentials',
  'Inverse Design',
  'Liquid-Phase Exfoliation',
  'Machine Learning in Materials Science',
  'Magn

In [34]:
doc.metadata

{'source': './data/data_rag/conference/89_AI4AM2024_Botti.pdf',
 'page': 0,
 'topic': ['Band Gap Engineering',
  'Machine Learning in Materials Science',
  'Exchange-Correlation Functionals',
  'Density Functional Theory (DFT)',
  'Computational Materials Science']}

In [35]:
for doc in documents:
    print(len(doc.metadata['topic']), doc.metadata['topic'])

5 ['Deep Learning', 'High Resolution Transmission Electron Microscopy (HRTEM)', 'Simulated Imaging', 'Nanoparticle Characterization', 'Deep Learning Algorithms for Image Processing']
5 ['Statistical Mechanics', 'Machine Learning', 'Interatomic Potentials', 'Computational Chemistry', 'Phase Diagrams']
1 ['Machine Learning and Deep Learning']
4 ['Machine Learning and Deep Learning', 'Computational Chemistry', 'Electrochemical Devices', 'Interatomic Potentials']
3 ['Machine Learning and Deep Learning', 'Computational Chemistry', 'Interatomic Potentials']
3 ['Computational Chemistry', 'Electrochemical Devices', 'Interatomic Potentials']
4 ['Machine Learning and Deep Learning', 'Computational Chemistry', 'Statistical Mechanics', 'Interatomic Potentials']
3 ['Machine Learning and Deep Learning', 'Computational Chemistry', 'Interatomic Potentials']
3 ['Machine Learning and Deep Learning', 'Computational Chemistry', 'Interatomic Potentials']
4 ['Computational Chemistry', 'Nanoparticle Characte

In [36]:
dict_topics["Graph Neural Networks"]

{'description': 'A type of neural network designed to work with data structured as graphs, useful for tasks involving relationships and connections between entities.',
 'metadata': [{'source': './data/data_rag/conference/17_AI4AM2024_Riu_Vicente_Jordi_67.pdf',
   'page': 0,
   'topic': ['ZX-Calculus',
    'Quantum Mechanics',
    'Quantum Circuit Optimization',
    'Reinforcement Learning',
    'Graph Neural Networks']},
  {'source': './data/data_rag/conference/17_AI4AM2024_Riu_Vicente_Jordi_67.pdf',
   'page': 1,
   'topic': ['Graph Neural Networks',
    'Reinforcement Learning',
    'ZX-Calculus',
    'Quantum Circuit Optimization']},
  {'source': './data/data_rag/conference/87_AI4AM2024_Abio_Albert_88.pdf',
   'page': 0,
   'topic': ['Finite Element Method (FEM)',
    'Surrogate Models',
    'Data Acquisition and Processing',
    'Reinforcement Learning',
    'Graph Neural Networks']},
  {'source': './data/data_rag/conference/87_AI4AM2024_Abio_Albert_88.pdf',
   'page': 0,
   'topic

- chunk size 5000 tokens gives 163 documents 
- N topics < 10
#####  result 68 topics

In [37]:
with open("AI4AM_topics2.json", "w+") as f:
    json.dump(dict_topics, f, indent=4)